# DigiPen Campus Agent

<a target="_blank" href="https://colab.research.google.com/github/simonguest/CS-394/blob/main/src/03/notebooks/campus-agent.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>
<a target="_blank" href="https://github.com/simonguest/CS-394/raw/refs/heads/main/src/03/notebooks/campus-agent.ipynb">
  <img src="https://img.shields.io/badge/Download_.ipynb-blue" alt="Download .ipynb"/>
</a>

## Install pre-requisites

In [ ]:
!uv pip install openai-agents==0.4.2 gradio==5.49.1

Using Python 3.12.12 environment at: /usr
Audited 2 packages in 63ms


## Set the OpenAI API Key Environment Variable

In [ ]:
import sys
import os
from dotenv import load_dotenv

if 'google.colab' in sys.modules:
  from google.colab import userdata # type:ignore
  OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
  os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
else:
  load_dotenv()

## Agents

In [ ]:
from agents import Agent, Runner, FileSearchTool, WebSearchTool

### Cafe Agent

In [ ]:
from agents import function_tool

@function_tool
def get_bytes_cafe_menu(date: str) -> any:
    return {
        f"{date}": {
            "daily byte": {
                "name": "Steak Quesadilla",
                "price": 12,
                "description": "Flank steak, mixed cheese in a flour tortilla served with air fried potatoes, sour cream and salsa",
            },
            "vegetarian": {
                "name": "Impossible Quesadilla",
                "price": 12,
                "description": "Impossible plant based product, mixed cheese in a flour tortilla served with air fried potatoes, sour cream and salsa",
            },
            "international": {
                "name": "Chicken Curry",
                "price": 12,
                "description": "Chicken thighs, onion, carrot, potato, curry sauce served over rice",
            },
        }
    }

In [ ]:
from datetime import datetime
from zoneinfo import ZoneInfo

cafe_agent = Agent(
    name="Cafe Agent",
    instructions=f"You help students locate and provide information about the Bytes Cafe. Get information in the official website: https://digipenbytes.myshopify.com/. Today is {datetime.now(ZoneInfo("America/Los_Angeles")).strftime("%A")}",
    tools=[WebSearchTool()]
)

I got hungry while working on this assignment, so I updated the hardcoded cafe data to fetch the latest information from the website.

### Set Vector Store

In [ ]:
VECTOR_STORE_ID = "vs_69740379d0fc819180d226886a532a51"

### Building Agent

In [ ]:
building_agent = Agent(
    name="Building Agent",
    instructions="You help students locate and provide information about buildings and rooms on campus. Be descriptive when giving locations.",
    tools=[
        FileSearchTool(
            max_num_results=3,
            vector_store_ids=[VECTOR_STORE_ID],
            include_search_results=True,
        )
    ],
)

### Course Agent

In [ ]:
course_agent = Agent(
    name="Course Agent",
    instructions="You help students find out information about courses held at DigiPen.",
    tools=[
        FileSearchTool(
            max_num_results=5,
            vector_store_ids=[VECTOR_STORE_ID],
            include_search_results=True,
        )
    ],
)

### Handbook Agent

In [ ]:
handbook_agent = Agent(
    name="Handbook Agent",
    instructions="You help students navigate the school handbook, providing information about campus policies and student conduct.",
    tools=[
        FileSearchTool(
            max_num_results=5,
            vector_store_ids=[VECTOR_STORE_ID],
            include_search_results=True,
        )
    ],
)

### Internship Agent

In [ ]:
##internship_agent = Agent(
    name="Internship Agent",
    instructions=f"You are an agent providing most up-to-date internship information for DigiPen students. DigiPen students primarily aim to find employment in game-related industries, and in particular, you must find internship information on the internet and inform DigiPen students majoring in Computer Science. Today is {datetime.now(ZoneInfo("America/Los_Angeles")).date()}",
    tools=[
        WebSearchTool()
    ],
)

I added an agent that collects internship information for CS students from the internet. I did this because I thought it would be very helpful for students like me who are currently job hunting. I configured it to refer to the internet for this purpose, since internship-related information requires finding the latest data. Additionally, I passed today's date to the AI so it can locate the most recent information.

### Campus Agent

In [ ]:
agent = Agent(
    name="DigiPen Campus Agent",
    instructions="You are a helpful campus agent that can plan and execute tasks for students at DigiPen. Please be concise and accurate in handing off tasks to other agents as needed.",
    handoffs=[building_agent, course_agent, handbook_agent, cafe_agent, internship_agent],
)

## Gradio Interface

### Chat function and display of tool calls

In [ ]:
from gradio import ChatMessage

async def chat_with_agent(user_msg: str, history: list):
    messages = [{"role": msg["role"], "content": msg["content"]} for msg in history]
    messages.append({"role": "user", "content": user_msg})
    responses = []
    reply_created = False
    active_agent = None

    result = Runner.run_streamed(agent, messages)
    async for event in result.stream_events():
        if event.type == "raw_response_event":
            if event.data.type == "response.output_text.delta":
                if not reply_created:
                    responses.append(ChatMessage(role="assistant", content=""))
                    reply_created = True
                responses[-1].content += event.data.delta
        elif event.type == "agent_updated_stream_event":
            active_agent = event.new_agent.name
            responses.append(
                ChatMessage(
                    content=event.new_agent.name,
                    metadata={"title": "Agent Now Running", "id": active_agent},
                )
            )
        elif event.type == "run_item_stream_event":
            if event.item.type == "tool_call_item":
                if event.item.raw_item.type == "file_search_call":
                    responses.append(
                        ChatMessage(
                            content=f"Query used: {event.item.raw_item.queries}",
                            metadata={
                                "title": "File Search Completed",
                                "parent_id": active_agent,
                            },
                        )
                    )
                else:
                    tool_name = getattr(event.item.raw_item, "name", "unknown_tool")
                    tool_args = getattr(event.item.raw_item, "arguments", {})
                    responses.append(
                        ChatMessage(
                            content=f"Calling tool {tool_name} with arguments {tool_args}",
                            metadata={"title": "Tool Call", "parent_id": active_agent},
                        )
                    )
            if event.item.type == "tool_call_output_item":
                responses.append(
                    ChatMessage(
                        content=f"Tool output: '{event.item.raw_item['output']}'",
                        metadata={"title": "Tool Output", "parent_id": active_agent},
                    )
                )
            if event.item.type == "handoff_call_item":
                responses.append(
                    ChatMessage(
                        content=f"Name: {event.item.raw_item.name}",
                        metadata={
                            "title": "Handing Off Request",
                            "parent_id": active_agent,
                        },
                    )
                )
        yield responses

### Launch Gradio

In [ ]:
import gradio as gr

demo = gr.ChatInterface(
    chat_with_agent,
    title="DigiPen Campus Agent",
    theme=gr.themes.Soft(
        primary_hue="red", secondary_hue="slate", font=[gr.themes.GoogleFont("Inter")]
    ),
    examples=[
        "I'm trying to find the WANIC classrooms. Can you help?",
        "What's the policy for eating in auditoriums?",
        "What's today's vegetarian dish at the Bytes Cafe?",
        "What are the prerequisites for FLM201?"
    ],
    submit_btn=True,
    flagging_mode="manual",
    flagging_options=["Like", "Spam", "Inappropriate", "Other"],
    type="messages",
    save_history=False,
)

demo.launch(share=False, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

#### Test Session

[user]

What's today's vegetarian dish at the Bytes Cafe?

[assistant]

Here’s today's vegetarian (Veggie Byte) dish at Bytes Cafe for Thursday, January 30, 2026:

- Veggie Byte: Fried Plant Based Chikin Sandwich — includes a plant‑based “chikin” patty, mayo, and pickles in a potato bun, served with potato salad (Thursday’s Veggie Byte) (digipenbytes.myshopify.com).

So if you're looking for a plant-based vegetarian option today, that's what’s available!
</assistant>

[user]

What's the internship situation like these days?

[assistant]

Here’s the current landscape for internship opportunities relevant to Computer Science majors at DigiPen in the game development and tech industry as of January 29, 2026—grounded in the latest data and resources:

DigiPen's Career Support & Internship Ecosystem

Handshake Platform

DigiPen students and alumni have exclusive access to Handshake, where you can post your resume, explore internship postings, and RSVP for recruiting events tailored to the video game industry. Company recruiters frequently use this platform. (digipen.edu)

Company Days & NextGen Events

DigiPen regularly hosts Company Days, bringing recruiters from top studios (e.g., Nintendo, Epic Games) to campus for presentations and interviews. Upcoming event details can be found on the Events Calendar. (digipen.edu)

NextGen 2026, the annual showcase connecting students with employers and alumni, is opening registration soon. It’s a prime opportunity to present your portfolio and build industry connections. (digipen.edu)

Internship for Credit & Advising Services

DigiPen facilitates both full-time and part-time paid internships through its Internship for Credit program, provided students meet prerequisites and receive pre-approval. (digipen.edu)

Ongoing one-on-one advising, resume and portfolio reviews, and mock interviews are available to help you prepare and stand out. (digipen.edu)

Recent DigiPen Internship Fair Highlights

On January 29, 2026, DigiPen’s annual online Internship Fair connected over 360 students with representatives from 21 game and tech companies, including big names like Epic Games, Respawn Entertainment, Santa Monica Studio, and Sucker Punch Productions. (digipen.edu)

The online format via Handshake enhanced scheduling efficiency, allowing students to pre-book group sessions and one-on-one meetings. (digipen.edu)

Notable 2026 Internship Listings (Game Industry)

Here are some of the high-profile internship programs currently open for application:

Activision Blizzard — Game Design & Game Engineering Internships
Multiple roles including Design (Call of Duty, Hearthstone, Overwatch, Diablo Immortal), Animation Engineering, Gameplay Engineering, Online Technology, Pipeline Engineering, Tools and UI Engineering.
Requirements typically include enrollment through July 2027, proficiency in C++, game engine experience, and availability for a 12-week, in-person internship starting late May to mid-June 2026.

These internships include relocation support and living stipends. Salary range: $20–$50/hour. (career.ucla.edu)

Nex — Software Engineer Intern (Game Development)

Based in San Jose, CA (hybrid model), this position runs from June 1 to August 31, 2026.

Responsibilities include developing motion-based games using a proprietary motion engine. Compensation ranges from $30–$40/hour. (builtinsf.com)

Second Dinner Studios — Product Manager Intern
Fully remote (with optional in-office time in Irvine) 12–13 week program, targeting mobile game product management. Compensation: $45–$50/hour. Application window ended December 21, 2025, but this reflects market context. (capd.mit.edu)

Broader Industry Context

Layoffs in the Game Industry (2022–2025)

The game industry experienced widespread layoffs, peaking in early 2024, with over 45,000 jobs lost by mid‑2025. This affected both major studios and smaller developers, emphasizing the competitive nature of current internship searches. (en.wikipedia.org)

Upcoming Game Developers Conference (GDC 2026: GDC Festival of Gaming)

Scheduled for March 9–13, 2026 in San Francisco, this rebranded event (now the “GDC Festival of Gaming”) offers extensive opportunities for networking, portfolio showcases, and employer engagement. A great place to scout emerging internship roles and future full-time opportunities. (en.wikipedia.org)

Summary & Action Plan

Use DigiPen’s Career Resources

Activate your Handshake profile, attend Company Days, and watch for NextGen 2026 registration to make meaningful connections.

Explore Major Internship Programs Now

Apply early to Activision Blizzard roles, Nex, and others that align with your interests and skills.

Prepare Strategically

Polish your resume, code samples, and portfolio. Leverage one-on-one advising and workshops to refine your presentation.

Engage at Industry Events

Attend GDC 2026 to maximize your exposure and learn about emerging opportunities.

Final Thoughts

Internship opportunities for DigiPen Computer Science students are robust right now—supported by institutional resources, major studio openings, and innovative startups. The competitive market highlights the importance of proactive preparation and networking.

Feel free to ask for help customizing application materials, selecting roles to target, or preparing for interviews—I'm here to support you every step of the way!

## Conclusion

In this module, I created an internship agent to add a new agent to DigiPen. To implement this, I granted it internet search permissions so that, rather than simply relying on information I prepared in advance, it could find relevant information on its own. I also provided the agent with the current date and DigiPen student details, allowing it to obtain newer and fresher information.

What was quite surprising was that it didn't take very long to get an answer; I was quite surprised that it found this much information with a latency of only 2-3 seconds.